# Импорт библиотек

import pickle
import random
import time
import os
import shutil
import copy

import numpy as np
import pandas as pd

from skimage import io
from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path
#import torchvision.transforms as transforms

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from __future__ import print_function, division
from matplotlib import colors, pyplot as plt
%matplotlib inline
plt.ion()

import torch
import torch.nn as nn
import torchvision.models as models
import torchsummary
from torch.utils.data import datasets, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.autograd import Variable
import torch.onnx

In [1]:
import pandas as pd
from tqdm.notebook import trange, tqdm_notebook
import torch
from torch.utils.data import DataLoader
import numpy as np
import torchvision
import random
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.onnx

In [2]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
EPOCHS = 40
BATCH_SIZE = 64
LEARNING_RATE = 0.001

SEED = 69
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

data_dir = './data/classed_nrj_train/'
test_dir = './data/test2/'

cuda


## Модель `cnn_v2`

In [3]:
#Набор трансформаций для обучающей выборки
train_transforms = transforms.Compose([transforms.CenterCrop(192), 
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.RandomRotation(45),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])
                                       ])

#Набор трансформаций для валидационной выборки
valid_transforms = transforms.Compose([transforms.CenterCrop(192),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                       ])

#Набор трансформаций для тестовой выборки
test_transforms = transforms.Compose([transforms.CenterCrop(192),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                      ])

In [4]:
#Объявляю функцию загрузчика с аргументами:
#Директория, Размер батча, Набор трансформаций для тренировочной выборки,
#Набор трансформаций для валидационной выборки, Размер валидационной выборки
def load_split_train_valid(datadir,
                           batch_size,
                           train_transforms,
                           valid_transforms,
                           valid_size): 
    
    #Загрузчик для тренировочной выборки
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    #Загрузчик для тренировочной выборки
    val_data = datasets.ImageFolder(datadir,
                    transform=valid_transforms)
    #Код для разделения на трейн и тест в указанном соотношении
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    trainloader = DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    valloader = DataLoader(val_data,
                   sampler=val_sampler, batch_size=batch_size)
    return trainloader, valloader

In [5]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_valid(datadir=data_dir,
                           batch_size=BATCH_SIZE,
                           train_transforms=train_transforms,
                           valid_transforms=valid_transforms,
                           valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))

['1', '10', '20', '3', '30', '6']
['1', '10', '20', '3', '30', '6']
166
42


In [6]:
class ConvNet(nn.Module): 
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop1 = nn.Dropout2d(p=0.25)
            self.conv2 = nn.Sequential(
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop2 = nn.Dropout2d(p=0.25)
            self.conv3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop3 = nn.Dropout2d(p=0.25)
            self.conv4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.fc1 = nn.Sequential(
                nn.Linear(12800, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
            )
            self.fc2 = nn.Sequential(
                nn.Linear(2048, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
            )
            self.out = nn.Linear(1024, 6)
        
        def forward(self, x):
            # x = self.drop1(self.conv1(x))
            # x = self.drop2(self.conv2(x))
            # x = self.drop3(self.conv3(x))
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.conv3(x)
            x = self.conv4(x)
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.out(x)

            return F.log_softmax(x, dim=1)

In [7]:
net = ConvNet()
net = net.cuda()

#Задаю функцию потерь
criterion = nn.NLLLoss()


#Задаю оптимизатор
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.5)

for epoch in trange(EPOCHS):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for i, (images, labels) in enumerate(tqdm_notebook(train_loader)):
        #Прямой запуск
        images=images.to(device)
        labels=labels.to(device)            
        outputs = net(images)
        loss = criterion(outputs, labels)

        #Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        
        running_loss += loss.item() * total
        running_corrects += (predicted == labels).sum().item()
        processed_data += total
    scheduler.step()

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, EPOCHS, running_loss / processed_data,
                  (running_corrects / processed_data) * 100))

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [1/40], Loss: 0.1980, Accuracy: 92.36%


  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [2/40], Loss: 0.0267, Accuracy: 99.17%


...

Epoch [38/40], Loss: 0.0036, Accuracy: 99.87%


  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [39/40], Loss: 0.0039, Accuracy: 99.88%


  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [40/40], Loss: 0.0024, Accuracy: 99.91%


In [8]:
# Epoch [40/40], Loss: 0.0081, Accuracy: 99.76%
# Epoch [10/10], Loss: 0.0152, Accuracy: 99.56%
# Epoch [10/10], Loss: 0.0066, Accuracy: 99.80%

In [9]:
#Проверка на валидационной выборке
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm_notebook(val_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Val Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

  0%|          | 0/42 [00:00<?, ?it/s]

Val Accuracy of the model on the test images: 99.96234939759037 %


In [10]:
# 99.96234939759037 %
# 99.5105421686747 %

## Submission

In [11]:
test_data = datasets.ImageFolder(test_dir,
                transform=test_transforms)

test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)
filenames = [line[0].split('\\')[1].split('.')[0] for line in test_data.imgs]
pred2class = [float(n) for n in train_loader.dataset.classes]
dict_pred = defaultdict(list)
dict_pred["id"] = filenames

In [12]:
%%time
net.eval()
with torch.no_grad():
    for i, (img, name) in enumerate(iter(test_loader)):
        img = img.to(device)
        outputs = net(img)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu()
        predicted = [pred2class[pred] for pred in predicted]
        dict_pred["regression_predictions"].extend(predicted)

Wall time: 1min 6s


In [13]:
data_frame = pd.DataFrame(dict_pred, columns=["id", "regression_predictions"])
data_frame.to_csv('./submissions/cnn-v4-nrj.csv', index=False, header=True)